<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Bot Financiero con Aprendizaje Reforzado</center></span>

<center>Entrenamiento de una agente para hacer algorithm trading</center>

<figure>
<center><img src="../Imagenes/trading.jpeg" width="400" height="300" align="center"/>
<figcaption>
<p style="text-align:center">Algorithmic trading</p>
</figcaption>
</center> 
</figure>

Fuente: [Money control](https://www.moneycontrol.com/news/business/markets/explained-what-is-algo-trading-and-why-sebi-wants-to-regulate-retail-investor-activity-in-this-segment-7812111.html)

##   <span style="color:blue">Autores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

## <span style="color:blue">Referencias</span>

1. [Alvaro Montenegro y Daniel Montenegro, Inteligencia Artificial y Aprendizaje Profundo, 2021](https://github.com/AprendizajeProfundo/Diplomado)
1. [Maxim Lapan, Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)
1. [Adaptado de Rowel Atienza, Advance Deep Learning with Tensorflow 2 and Keras,Pack,2020](https://www.amazon.com/-/es/Rowel-Atienza-ebook/dp/B0851D5YQQ).
1. [Sutton, R. S., & Barto, A. G. (2018).Reinforcement learning: An introductio, MIT Press, 2018](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf)
1. [Ejecutar en Colab](https://colab.research.google.com/drive/1ExE__T9e2dMDKbxrJfgp8jP0So8umC-A#sandboxMode=true&scrollTo=2XelFhSJGWGX)
1. [Human-level control through deep reinforcement
learning](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Los datos](#Los-datos)
* [Gráfico de velas](#Gráfico-de-velas)
* [Descripción del problema y primeras decisiones](#Descripción-del-problema-y-primeras-decisiones)
* [Librería aplr](#Librería-aplr)
* [Librería trading](#Librería-trading)
* [Modelos-Dueling](#Modelos-Dueling)
* [Implementación de los Modelos](#Implementación-de-los-Modelos)
* [Entrenamiento del modelo](#Entrenamiento-del-modelo)


## <span style="color:blue">Introducción</span>

En lección entrenaremos una agente capaz de invertir en la bolsa. Para desarrollar este proyecto, nos hemos basado en la lección 10 del libro [Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67).

El repositorio del libro puede encontrarlo en [Github](https://github.com/PacktPublishing/Deep-Reinforcement-Learning-Hands-On). Allí encuentra los datosa y ejemplos. También puede instalar si lo prefiere  la librería [PTAN](https://github.com/Shmuma/ptan) del mismo autor.

Paea esta lección hemos extraido algunos módulos de la lección y hemos construido nuestro módulo `aplr`, en el cual hemos colocado las herramientas que usaremos y que describimos en las siguientes secciones.

## <span style="color:blue">Los datos</span>

En nuestro ejemplo, utilizaremos los precios del mercado de valores ruso del período 2015-2016. 

Dentro del archivo, tenemos archivos CSV con barras M1, lo que significa que cada fila
en cada archivo CSV corresponde a un solo minuto en el tiempo, y el movimiento del precio
durante ese minuto se captura con cuatro precios: apertura, máximo, mínimo y cierre. Aquí,
un precio de apertura es el precio al comienzo del minuto, alto es el máximo
precio durante el intervalo, bajo es el precio mínimo, y el precio de cierre es el
último precio del intervalo de tiempo minuto. 

Cada intervalo de un minuto se llama barra y nos permite tener una idea del movimiento del precio dentro del intervalo. por ejemplo, en el archivo YNDX_160101_161231.csv (que tiene acciones de la empresa Yandex para 2016), tenemos 130k líneas en este formulario:

## <span style="color:blue">Gráfico de velas</span>

La forma típica de representar esos precios se llama gráfico de velas japonesas, donde cada
la barra se muestra como una vela. Parte de las cotizaciones de Yandex para un día en febrero de 2016 se muestra en el siguiente cuadro. La carpeta de datos contiene dos archivos con datos M1 para 2016 y 2015. Usaremos datos de 2016 para entrenamiento de modelos y datos de 2015 para
validación.

<figure>
<center><img src="../Imagenes/candelabro.png" width="600" height="600" align="center"/>
<figcaption>
<p style="text-align:center">Gráfico de velas</p>
</figcaption>
</center> 
</figure>

Fuente: [Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020, pag 261](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)

## <span style="color:blue">Descripción del problema y primeras decisiones</span>

Implementaremos el agente para trading básico básico en su forma más simple. La observación incluirá la siguiente información:

* N barras pasadas, donde cada una tiene precios de apertura, máximo, mínimo y cierre
* Una indicación de que la acción fue comprada hace algún tiempo (solo una acción en un tiempo será posible)
* Ganancia o pérdida que tenemos actualmente de nuestra posición actual (la participación comprada)

En cada paso, después de cada barra de minutos, el agente puede tomar una de las siguientes
comportamiento:

* **No hacer nada**: saltar la barra sin realizar ninguna acción.
* **Comprar una acción**: si el agente ya tiene la acción, no se comprará nada; de lo contrario, pagaremos la comisión, que suele ser una pequeña porcentaje del precio actual.
* **Cerrar la posición**: si no tenemos una acción comprada  previamente, nada pasará; de lo contrario, pagaremos la comisión por la transacción


La `recompensa` que recibe el agente se puede expresar de varias formas. Por una lado, podemos dividir la recompensa en varios pasos durante nuestra propiedad de la acción. En ese caso, la recompensa en cada paso será igual al movimiento de la última barra. 

Por el otro lado, el agente recibirá la recompensa solo después de cerrar la acción y
recibir la recompensa completa a la vez. A primera vista, ambas variantes deberían tener el mismo resultado final, pero tal vez con diferentes velocidades de convergencia. Sin embargo, en la práctica, la diferencia podría ser drástica.

Una última decisión a tomar es cómo representar los precios en las observaciones del ambiente. Idealmente, nos gustaría que nuestro agente fuera independiente del precio real
valores y tener en cuenta el movimiento relativo, como "la acción ha crecido un 1%
durante la última barra" o "la acción ha perdido un 5%". Esto tiene sentido, ya que
los precios pueden variar, pero pueden tener patrones de movimiento similares.

Para lograr esto, convertiremos los valores de apertura, alto, bajo y y precios de cierre a tres números que muestran precios máximos, mínimos y de cierre representados como un porcentaje del precio de apertura.

Esta decisión tiene algunos inconvenientes prácticos como el hecho que a veces los precios se quedan pegados por un tiempo al llegar a cifras redondas, como  $500.000,oo, lo cual no podría detectar el agente. Pero por otro lado esta es una decisión muy conveniente para facilitarle el trabajo a la red neuronal.

## <span style="color:blue">Librería aplr</span>

Escribir módulos en Python es muy sencillo. Para crear un módulo propio, simplemente creamos un nuevo archivo . py con el nombre del módulo, y después lo importamos usando el nombre del archivo Python (sin la extensión `.py`) usando el comando `import`.

El primer módulo será su librería. En nuestro caso, creamo al carpeta `aplr`que será nuestra librería.

Para construir su propia librería, lo que tiene que hacer es lo siguiente. 

1. Cree una carpeta en el sitio de su preferencia. Nosotros hemos creado el módulo dentro de la carpeta de Cuadernos de este tutorial, para facilitar el acceso directo a la librería. Sin embargo para hacer su deploy, Puede ser necesario crear una acceso en el path de Python, para tener una acceso más genérico. Para los detalles puede cosnultar cualquier tutorial por ejemplo [learnpython.org](https://www.learnpython.org/es/Modules%20and%20Packages#:~:text=Escribiendo%20módulos,py)
2. Dentro de la carpeta cree un archivo vacío con el nombre `__init__.py`. 
3. Coloque los archivos con extensión `py` que conforman parte de su librería. Por ejemplo hemos colocado los archivos `actions.py`, `agent.py`, .... que serán nuestros módulos baśicos.
4. Si desea crear un sublibrería, cree la carpeta dentro de la carpeta de su librería principal, agregue allí un archivo vacío con el nombre `__init__.py` y agrege sus archivos `.py`. Por ejemplo, para este tutorial hemos creado la librería  `trading` dentro de `aprl` y allí hemos colocado los archivos que representan los módulos específicos para este tutorial de trainig.

Ya estamos listos para importar los objetos que vamos a usar. Primero importamos los módulos y submódulos  y luego los archivos específicos. Por ejemplo:

A continuación describimos los módulos que usaremos en este ejercicio.

### Módulo actions

Este módulo implementa una jerarquía de clases para dar acceso a las diferentes formas de seleccionar acciones que hemos estudiado. Las clases disponibles son

- **ActionSelector**: Clase abstracta que convierte puntuaciones (scores) en acciones, para un lote scores.
- **ArgmaxActionSelector**: Selecciona acciones usando argmax.
- **EpsilonGreedyActionSelector**: Selecciona acciones usando epsilon-gredy.
- **ProbabilityActionSelector**: Convierte probabilidades de acciones en acciones al muestrearlas.
- **EpsilonTracker**: Actualiza epsilon de acuerdo con un horario lineal.

### Módulo agent

Este módulo implementa una jerarquía de clases para dar acceso a las diferentes formas de seleccionar acciones que hemos estudiado. Las clases disponibles son:

- **BaseAgent**: Interfaz abstracta de una agente. incluye la creación de un estado inicial del agente y la función __call__ para cuando es llamado. Ambas deben implementarse en clase derivadas.
- **DQNAgent**: es un agente DQN sin memoria que calcula valores Q de las observaciones y las convierte en acciones usando action_selector
-  **TargetNet**: Envoltura (Wrapper) alrededor del modelo que proporciona una copia en lugar de pesos entrenadas.
- **PolicyAgent**: espera que la red produzca una distribución de políticas en un conjunto discreto de acciones La distribución de políticas puede ser logits (no normalizada) o normalizada. distribución. En la práctica, siempre debe usar logits para mejorar la estabilidad numérica  del proceso de entrenamiento.
- **ActorCriticAgent** agente para el método actor-crítico que estudiamos en la siguiente lección

Los siguientes métodos utilitarios son incluidos en el módulo:

- *default_states_preprocessor*:   Convierte la lista de estados en la forma adecuada para el modelo.
- *float32_preprocessor*: transforma los valores de los estados en float32
.

### Módulo experience

La abstracción de agentes descrita en la sección anterior nos permite implementar
comunicaciones del entorno de forma genérica. Estas comunicaciones ocurren en la
forma de trayectorias, producidas al aplicar las acciones del agente al entorno de Gym.

En un alto nivel, las clases de fuente de experiencia toman la instancia y el entorno del agente
y proporciona los datos paso a paso de las trayectorias. La funcionalidad
de esas clases incluye:

* Soporte de múltiples entornos que se comunican al mismo tiempo. Esto permite una utilización eficiente de la GPU a medida que se realiza un lote de observaciones. procesados ​​por el agente a la vez.
* Una trayectoria se puede preprocesar y presentar en una forma conveniente para el entrenamiento. Por ejemplo, hay una implementación de lananzamiento de  subtrayectoria con acumulación de la recompensa. 
* Compatibilidad con entornos vectorizados de OpenAI Universe


Hay tres clases proporcionadas por el módulo:

* **ExperienceSource**: utilizando el agente y el conjunto de entornos, produce subtrayectorias de n pasos con todos los pasos intermedios
* **ExperienceSourceFirstLast**: esto es lo mismo que ExperienceSource, pero
* **ExperienceSourceRollouts**: sigue la ventaja asíncrona en lugar de una subtrayectoria completa (con todos los pasos), mantiene solo el primero y el último pasos, con la acumulación adecuada de recompensas en el medio. 

Todas las clases están escritas para ser eficientes tanto en términos de unidad central de procesamiento
(CPU) y la memoria, que no es muy importante para los problemas de juguete, pero podría volverse
un problema cuando quiere resolver juegos de Atari y necesitas mantener 10 millones de muestras en el
búfer de reproducción utilizando hardware básico.

## <span style="color:blue">Librería trading</span>

### Módulo data

Este módulo está diseñado para leer un archivo de datos con las siguientes columnas: 
- `<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>`

Los datos son anotados en una tupla nombrada:

Se tienen las siguientes funciones

- **read_csv**: lee un archivo de datos de tipo csv. Omite registros  cuando la diferencia entre el valor de entrada y los demás valores está pode debajo de 1e-8. Además asegura que el precio de apertura (open) de un registro coincida con el precio de cierre(close) del resgistro anterior.
- **prices_to_relative**: Convierte precios a relativos con respecto al precio de apertura: `(precio -precio_open)/precio_open`.
- **load_relative**: Lee el archivo y convierte a precios relativos
- **price_files**: construye una lista con los archivos de tipo `csv` encontrados en un directorio. Por ejemplo en los datos tenemos archivos separados para cada año: 2015 y 2016.
- **load_year_data**: determina el año de los datos a a partir del nombre de los archivos.

### Módulo environ

En este módulo se implementa un ambiente específico para nuestro problema.
La clase más importante del módulo es 

- **class StocksEnv(gym.Env)**: ESta la clase que implementa el ambiente y es derivada de la clase base gym.Env. El ambientese basa en gym.Env para aprovechar toda la implemtación básica de OpenAI gym.Env. 

Por su importancia vamos a describir con alg+un detalle la API pública de este módulo.


Las acciones posibles para el problemea son codificadas como campos de enumeración. 

Los atributos *metadata* y *spec* son requeridos para compatibilidad con *gym.Env*. No vamos a hacer ningún tipo de renderización, por lo que ignoraremso esto.

Esto método de clase permite tener dos constructores para la clase.  En este caso, por jemplo puede instanciar un objeto de tipo *StocksEnv* entregándole la dirección de la carpeta en donde estan los rachivos de datos. Puede usar este métod así

In [ ]:
El constructor básico de la clase es:

* *prices* : Contiene uno o más precios de acciones para uno o más instrumentos como un diccionario, donde las claves son el nombre del instrumento y el valor es un contenedor de objetos `data.Prices` , que contiene matrices de datos de precios.
* *bars_count* : El conteo de barras que pasamos en la observación. Por defecto, esto es 10 barras. Revise la gráfica abajo.
* *comission* : El porcentaje del precio de la acción que tenemos que pagar al intermediario (broker) en la compra y venta de acciones. Por defecto, es 0.1%.
* *reset_on_close*: si este parámetro se establece en *True*, por defecto es así, cada vez que el agente nos pide cerrar la posición existente (es decir,  vender una acción), detenemos el episodio. De lo contrario, el episodio continuará hasta que el final de nuestra serie de tiempo, que es un año de datos.
* *conv_1d* : este argumento booleano cambia entre diferentes representaciones  de datos de precios en la observación pasada al agente. Si se establece en *True*, observaciones tienen una forma 2D, con diferentes componentes de precio para barras posteriores organizadas en filas. Por ejemplo, precios altos (high) (precio máximo para la barra) se colocan en la primera fila, los precios bajos (low) en la segunda y  precios de cierre (close) en el tercero. Esta representación es adecuada para hacer convolución 1D en series de tiempo, donde cada fila en los datos tiene el mismo significado que diferentes planos de color (rojo, verde o azul) en imágenes 2D de Atari. si establecemos esta opción a False, tenemos una única matriz de datos con cada barra componentes colocados juntos. Esta organización es conveniente para una completa arquitectura de red conectada. Ambas representaciones se ilustran en figura abajo.
* *random_ofs_on_reset*: si el parámetro es *True*, lo es por defecto, en cada reinicio del entorno, se elegirá un desplazamiento aleatorio en la serie temporal. De lo contrario, comenzaremos desde el principio de los datos.
* *reward_on_close* : este parámetro booleano cambia entre  dos esquemas de recompensa: luego de cada iteración, o,  al final del episodio. Si se establece en *True* , el agente reciba una recompensa solo con la  acción "close". De lo contrario, le daremos un pequeña recompensa cada barra, correspondiente al movimiento del precio durante esa barra.
* *volumes* : este argumento activa los volúmenes en las observaciones y es deshabilitado por defecto.

<figure>
<center><img src="../Imagenes/data_representation_trading.png" width="400" height="300" align="center"/>
<figcaption>
<p style="text-align:center">Represntacion de los datos para diferentes arquitecturas de  la red</p>
</figcaption>
</center> 
</figure>

Fuente: [Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020, pag 265](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)

El cuerpo del constructor continua así:

La mayor parte de la funcionalidad de la clase `StocksEnv` está implementada en las clases 

- **State**: prepara los datos como una lista de barras (high, low, close, volume). Observe la imagen arriba.
- **State1D**: Prepara los datos en forma de matriz. Como se ve en la imagen arriba.

El constructor crea los datos de acuerdo con el parámetro *conv_1d*. Adicionalmente, el constructor crea crea el espacio de acciones y el de observaciones de acuerdo con la API de Gym.

El resto de la API es como sigue:

Este método define la funcionalidad *reset()* para nuestro entorno. De acuerdo a
a la semántica gym.Env, cambiamos aleatoriamente la serie de tiempo que trabajaremos
y seleccionamos el desplazamiento inicial en esta serie temporal. El precio seleccionado y el desplazamiento se pasan a la instancia de estado interno, que luego solicita una observación inicial usando su función *encode()*.

Este método tiene que manejar la acción elegida por el agente y devolver la siguiente observación, recompensa y marca de hecho. Toda la funcionalidad real está implementada en nuestro clases de estado, por lo que este método es un contenedor (wrapper) muy simple alrededor de la llamada al los métodos de estado de Gym.

La API de Gym permite definir el método *render()* para visualizar el proceso. El método *close()* es llamado por el ambiente para destruir todos los objetos libres. No haremos ninguna implementación de estos métodos de Gym.

Este método permite corregir problemas con el generador de números pseudo-aleatorios de Python, para las situaciones de múltiples ambientes simultáneos. No se usará aquí.

A continuación vamos a revisar brevemente la clase  *State*. La clase *State* es similar con los cambios correspondientes al manejo apropiado de los datos.

Esta clases son las encargadas del trabajo fuerte del ambiente. comencemos con el constructor.

Simplemente verifica los tipos de datos y los anota para recordarlos localmente.

El método *reset()* se llama cada vez que se le pide al entorno que se reinicie y tiene que guardar los datos de precios pasados  y la sl desplamiento inicial para empezar a leer datos. Al principio,  no se  tiene acciones compradas, por lo que se tiene *have_position=False* y *open_
precio=0.0*.

Esta propiedad retorna el tamaño de cada dato de *State* en un arreglo de Numpy. 

El siguiente método es el que hace la codificación (encode) de la observación para este caso un vector simple que incluye precios con opción volúmenes y dos números que indican la presencia de un activo comprado y ganancia por la  posición 


El método *_cur_close()* entrega el precio de cierre de la barra actual. Precios pasados a la clase *State* tienen la forma relativa con respecto al precio de apertura: el máximo, mínimo y cierre. Los componentes son proporciones relativas al precio de apertura. Esta representación ya se había discutido cuando hablamos sobre los datos de entrenamiento, y (probablemente) ayudará a nuestro agente para aprender patrones de precios que son independientes del valor del precio real.

Revisamos ahora, la pieza de código más complicada de la clase *State*. El método *step* el cual esresponsable de realizar un paso en nuestro entorno. A la salida, tiene que devolver la
recompensa en un porcentaje y una indicación del final del episodio.

In [ ]:
    def step(self, action):
        assert isinstance(action, Actions)
        reward = 0.0
        done = False
        close = self._cur_close()
        if action == Actions.Buy and not self.have_position:
            self.have_position = True
            self.open_price = close
            reward -= self.commission_perc

Si el agente ha decidido comprar una acción, cambiamos nuestro estado y pagamos la comisión.
En nuestro estado, asumimos la ejecución instantánea de la orden al precio de cierre de la barra actual,
lo cual es una simplificación por nuestra parte; normalmente, una orden se puede ejecutar en un
precio diferente, lo que se denomina deslizamiento de precios (price slippage).

Si tenemos una posición y el agente nos pide que la cerremos, volvemos a pagar comisión,
cambiamos la bandera done  si estamos en modo *reset_on_close*, y se otorga una recompensa final por toda la posición, y cambia nuestro estado.

En el resto de la función, modificamos el desplazamiento actual y damos la recompensa por el
movimiento de la última barra. Eso es todo para la clas *State*. Omitimos la calse *State1D* por ser similar.

### Módulo common

Este módulo contiene  la siguiente clase utilitaria.

- **RewardTracker**: Esta es una envoltura que calcula la recompensa completa para los últimos 100 episodios  y además indica cuando la recompensa promedio excede un umbral deseado.

Además incluye las funciones utilitarias

- *calc_values_of_states*: calcula el valor de los estados.
- *unpack_batch*: desempaqueta los objetos en los lotes (batches de datos).
- *calc_loss: Calcula la funcuón de pérdida.

### Módulo validation

Este módulo solamente contiene la función

- *validation_run*: corre un validación del modelo

### Módulo models

Contiene los modelos de redes neuronales presentados abajo.

## <span style="color:blue">Modelos-Dueling</span>

El modelo Dueling es una mejora para DQN fue propuesta en 2015, en el documento llamado Wang et al., 2015, Dueling Network Architectures for Deep Reinforcement Learning](https://arxiv.org/pdf/1511.06581.pdf). 

La observación central de este artículo es que los valores Q, Q(s, a), que nuestra red están intentando
para aproximar, se puede dividir en dos cantidades: el valor del estado, V(s), y el
ventaja de las acciones en este estado, A(s, a).
Hemos visto la cantidad V(s) antes, ya que era el núcleo de la iteración de valor.  Esta cantidad es es igual a la recompensa esperada descontada que se puede obtener de este estado. 

La ventaja A(s, a) se supone que cierra la brecha de A(s) a Q(s, a), ya que, por definición, Q(s, a) = V(s) +
A(s, a). En otras palabras, la ventaja A(s, a) es solo el delta, que indica cuánto extra
recompensa alguna acción particular del estado nos trae.



Un problema por resolver es que la suma puede ser hecha don muy diversos valores. Los estadísticos dicen que se tiene un problema de identificabilidad, porque hay infinitas soluciones. Es necesario establer una restricción que lleve auna única solución. 

Esta restricción podría aplicarse de varias maneras, por ejemplo, a través de la pérdida
función; pero en el artículo Dueling, los autores propusieron una muy elegante
solución de restar el valor medio de la ventaja de la expresión Q
en la red, lo que efectivamente lleva la media de la ventaja a cero:

$$
Q(s,a) = V(s) + A(s,a) - \frac{1}{N}\sum_k A(s,k).
$$

En los modelos de esta lección introducimos este modelo, lo cual es bastante simple implementado dos módulos dentro de la red, como se muestra en la siguiente imagen.

<figure>
<center><img src="../Imagenes/dueling.png" width="600" height="600" align="center"/>
<figcaption>
<p style="text-align:center">Modelo Dueling</p>
</figcaption>
</center> 
</figure>

Fuente: [Deep Reinforcement Learning Hands-On: Apply modern RL methods to practical problems of chatbots, robotics, discrete optimization, web automation, and more, 2nd Edition, 2020, pag 261](http://library.lol/main/F4D1A90C476A576238E8FE1F47602C67)

## <span style="color:blue">Implementación de Modelos</span>

## <span style="color:blue">Entrenamiento del modelo</span>

En el entrenamiento usamos una envoltura (wrapper): `RewardTraker`. Esta envoltura calcula la recompensa completa para los últimos 100 episodios  y además indica cuando la recompensa promedio excede un umbral deseado.

In [6]:
import os
import gym
#import ptan
#import argparse
import numpy as np

import torch
import torch.optim as optim

import aprl
from aprl import trading

from aprl.trading import environ, data, models, common, validation

from torch.utils.tensorboard import SummaryWriter

In [ ]:
BATCH_SIZE = 32
BARS_COUNT = 10
TARGET_NET_SYNC = 1000
DEFAULT_STOCKS = "data/YNDX_160101_161231.csv"
DEFAULT_VAL_STOCKS = "data/YNDX_150101_151231.csv"

GAMMA = 0.99

REPLAY_SIZE = 100000
REPLAY_INITIAL = 10000

REWARD_STEPS = 2

LEARNING_RATE = 0.0001

STATES_TO_EVALUATE = 1000
EVAL_EVERY_STEP = 1000

EPSILON_START = 1.0
EPSILON_STOP = 0.1
EPSILON_STEPS = 1000000

CHECKPOINT_EVERY_STEP = 1000000
VALIDATION_EVERY_STEP = 100000


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=False, action="store_true", help="Enable cuda")
    parser.add_argument("--data", default=DEFAULT_STOCKS, help="Stocks file or dir to train on, default=" + DEFAULT_STOCKS)
    parser.add_argument("--year", type=int, help="Year to be used for training, if specified, overrides --data option")
    parser.add_argument("--valdata", default=DEFAULT_VAL_STOCKS, help="Stocks data for validation, default=" + DEFAULT_VAL_STOCKS)
    parser.add_argument("-r", "--run", required=True, help="Run name")
    args = parser.parse_args()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    saves_path = os.path.join("saves", args.run)
    os.makedirs(saves_path, exist_ok=True)

    if args.year is not None or os.path.isfile(args.data):
        if args.year is not None:
            stock_data = data.load_year_data(args.year)
        else:
            stock_data = {"YNDX": data.load_relative(args.data)}
        env = environ.StocksEnv(stock_data, bars_count=BARS_COUNT, reset_on_close=True, state_1d=False, volumes=False)
        env_tst = environ.StocksEnv(stock_data, bars_count=BARS_COUNT, reset_on_close=True, state_1d=False)
    elif os.path.isdir(args.data):
        env = environ.StocksEnv.from_dir(args.data, bars_count=BARS_COUNT, reset_on_close=True, state_1d=False)
        env_tst = environ.StocksEnv.from_dir(args.data, bars_count=BARS_COUNT, reset_on_close=True, state_1d=False)
    else:
        raise RuntimeError("No data to train on")
    env = gym.wrappers.TimeLimit(env, max_episode_steps=1000)

    val_data = {"YNDX": data.load_relative(args.valdata)}
    env_val = environ.StocksEnv(val_data, bars_count=BARS_COUNT, reset_on_close=True, state_1d=False)

    writer = SummaryWriter(comment="-simple-" + args.run)
    net = models.SimpleFFDQN(env.observation_space.shape[0], env.action_space.n).to(device)
    tgt_net = agent.TargetNet(net)
    selector = actions.EpsilonGreedyActionSelector(EPSILON_START)
    agent = agent.DQNAgent(net, selector, device=device)
    exp_source = .experience.ExperienceSourceFirstLast(env, agent, GAMMA, steps_count=REWARD_STEPS)
    buffer = experience.ExperienceReplayBuffer(exp_source, REPLAY_SIZE)
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

    # main training loop
    step_idx = 0
    eval_states = None
    best_mean_val = None

    with common.RewardTracker(writer, np.inf, group_rewards=100) as reward_tracker:
        while True:
            step_idx += 1
            buffer.populate(1)
            selector.epsilon = max(EPSILON_STOP, EPSILON_START - step_idx / EPSILON_STEPS)

            new_rewards = exp_source.pop_rewards_steps()
            if new_rewards:
                reward_tracker.reward(new_rewards[0], step_idx, selector.epsilon)

            if len(buffer) < REPLAY_INITIAL:
                continue

            if eval_states is None:
                print("Initial buffer populated, start training")
                eval_states = buffer.sample(STATES_TO_EVALUATE)
                eval_states = [np.array(transition.state, copy=False) for transition in eval_states]
                eval_states = np.array(eval_states, copy=False)

            if step_idx % EVAL_EVERY_STEP == 0:
                mean_val = common.calc_values_of_states(eval_states, net, device=device)
                writer.add_scalar("values_mean", mean_val, step_idx)
                if best_mean_val is None or best_mean_val < mean_val:
                    if best_mean_val is not None:
                        print("%d: Best mean value updated %.3f -> %.3f" % (step_idx, best_mean_val, mean_val))
                    best_mean_val = mean_val
                    torch.save(net.state_dict(), os.path.join(saves_path, "mean_val-%.3f.data" % mean_val))

            optimizer.zero_grad()
            batch = buffer.sample(BATCH_SIZE)
            loss_v = common.calc_loss(batch, net, tgt_net.target_model, GAMMA ** REWARD_STEPS, device=device)
            loss_v.backward()
            optimizer.step()

            if step_idx % TARGET_NET_SYNC == 0:
                tgt_net.sync()

            if step_idx % CHECKPOINT_EVERY_STEP == 0:
                idx = step_idx // CHECKPOINT_EVERY_STEP
                torch.save(net.state_dict(), os.path.join(saves_path, "checkpoint-%3d.data" % idx))

            if step_idx % VALIDATION_EVERY_STEP == 0:
                res = validation.validation_run(env_tst, net, device=device)
                for key, val in res.items():
                    writer.add_scalar(key + "_test", val, step_idx)
                res = validation.validation_run(env_val, net, device=device)
                for key, val in res.items():
                    writer.add_scalar(key + "_val", val, step_idx)